# H A $\alpha$ decomposition evaluation

In this notebook we compare results of the new implementation of the
H A $\alpha$ decomposition with the C legacy version of PolSARpro.  
It assumes the user has a working installation of PolSARpro and has followed the 
instructions in the README.md file.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
# avoid thread conflicts between numpy and dask
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

from polsarpro.devtools import parse_psp_parameter_string
import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.io import read_psp_bin, read_T3_psp, open_netcdf_beam
from polsarpro.util import S_to_C2
from polsarpro.decompositions import h_a_alpha
import shutil

# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_convert/"

# change to your data paths
# original dataset
input_alos_data = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")

# input files from C
input_test_dir = Path("/data/psp/SAN_FRANCISCO_ALOS1/C2")

# output files from C
output_test_dir = Path("/data/psp/res/h_a_alpha_2x2_c")

## Run the C-version on some test data

In [ ]:
# fl1: alpha, beta, delta, lambda
# fl2: lambda
# fl3: alpha
# fl4: entropy 
# fl5: anisotropy
# fl6-9 combinations

input_str= f"""id: {input_test_dir} 
od: {output_test_dir}
iodf: C2
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
fl1: 1
fl2: 1
fl3: 1
fl4: 1
fl5: 1
fl6: 0
fl7: 0
fl8: 0
fl9: 0
errf: /tmp/MemoryAllocError.txt
mask: {input_test_dir}/mask_valid_pixels.bin
"""
result = parse_psp_parameter_string(input_str)
os.system(f"h_a_alpha_decomposition.exe {result}")

# CPSP does not create another config file in output dir
shutil.copy(input_test_dir / "config.txt", output_test_dir)

## Load ALOS data and C outputs

In [ ]:
# uncomment to test on S matrix made with SNAP
S = open_netcdf_beam(input_alos_data)

# uncomment to test on T3 matrix made with C-PSP
# T3 = read_T3_psp(input_test_dir)

# out_names = ("entropy", "anisotropy", "alpha")
out_names = ("entropy", "anisotropy", "alpha",  "delta", "lambda")

## Apply the decomposition

In [ ]:
file_out = "/data/psp/res/test_h_a_alpha.nc"
# netcdf writer cannot overwrite
if os.path.isfile(file_out):
    os.remove(file_out)

with ProgressBar():
    # uncomment to compute and store result in the notebook
    # out_py = h_a_alpha(S, boxcar_size=[7, 7], flags=out_names).compute()

    # uncomment to compute and store result in a file
    C2 = S_to_C2(S, p1="hh", p2="hv")
    res = h_a_alpha(C2, boxcar_size=[7, 7], flags=out_names).compute()#.to_netcdf(file_out)
    
    # uncomment to compute and store result in a file (T3 inputs)
    # h_a_alpha(T3, boxcar_size=[7, 7], flags=out_names).to_netcdf(file_out)

# Numerical evaluation

In [ ]:
# open python output (comment if using compute)
# out_py = xr.open_dataset("/data/psp/res/test_h_a_alpha.nc")
out_py = res

# open C-PolSARPro outputs
out_c = {}
for name in out_names:
    file_name = output_test_dir / f"{name}.bin"
    out_c[name] = read_psp_bin(file_name)

# compare C with python
for var in out_names:
    var_c =  out_c[var]
    var_da = out_py[var]
    relerr = np.nanmean(((np.abs(var_da - var_c) / np.abs(var_c + var_da))))
    print(f"Variable {var}")
    print(f"Relative error between C and python: {relerr}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.imshow(abs(out_py.delta[::8]-out_c['delta'][::8])>0.1, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)

plt.figure(figsize=(10, 10))
plt.imshow(out_py.delta[::8], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)

plt.figure(figsize=(10, 10))
plt.imshow(out_c['delta'][::8], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)